In [1]:
#요일별&시간대별 교통사고 데이터와 월별&요일별 데이터를 사용하여
#월별(계절), 요일별(주말, 공휴일 등), 시간대별(출퇴근시간)로 교통사고 어떻게 발생하는지 분석하는 것을 목표로 설정하였습니다. 
import pandas as pd

In [14]:
df=pd.read_csv("도로교통공단_요일별 시간대별 교통사고 통계_20221231.csv", encoding='cp949')
df.head()

,요일,시간대,사고건수,사망자수,중상자수,경상자수,부상신고자수
0,일,00시-02시,1208,21,331,1441,119
1,일,02시-04시,664,24,211,674,64
2,일,04시-06시,541,21,165,542,58
3,일,06시-08시,623,18,183,658,43
4,일,08시-10시,1132,26,339,1297,108
